In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Initializing weights 

In [4]:
def init_weights(shape):
    init_rand_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_rand_dist)

In [5]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [6]:
def conv2d(x,W):
    # x is input tensor [batch, H,W,Channels]
    # W is kernel
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [7]:
def max_pool_2_by_2(x):
    # x is input tensor [batch, H,W,Channels]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [8]:
def convolutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

In [9]:
def normal_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W) +b

In [10]:
x = tf.placeholder(tf.float32,shape=[None,784])
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [11]:
x_image = tf.reshape(x,[-1,28,28,1])

In [12]:
conv_1 = convolutional_layer(x_image,shape=[5,5,1,32])
conv_1_pool = max_pool_2_by_2(conv_1)

In [13]:
conv_2 = convolutional_layer(conv_1_pool,shape=[5,5,32,64])
conv_2_pool = max_pool_2_by_2(conv_2)

In [14]:
conv_2_flat = tf.reshape(conv_2_pool,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_layer(conv_2_flat,1024))

In [15]:
hold_prob = tf.placeholder(tf.float32) 
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [16]:
y_pred = normal_layer(full_one_dropout,10)

In [17]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [18]:
optmizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optmizer.minimize(cross_entropy)
steps = 5000

In [19]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        if i % 100 == 0:
            print('On Step {}'.format(i))
            print('Accuracy: ')
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            
            accuracy = tf.reduce_mean(tf.cast(matches,tf.float32))
            print(sess.run(accuracy,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            
            print('\n')

On Step 0
Accuracy: 
